## 26. When to add logging and how much

Logging is your runtime **black box recorder**—it answers *“What was the system doing right before things went wrong?”*  

Guidelines:
* **INFO**: high‑level application events (user login, job started/finished).
* **DEBUG**: developer details (SQL queries, loop counters) – verbose, disabled in prod.
* **WARNING/ERROR**: unexpected conditions that deserve attention.

Common pitfalls:
• Oversharing at INFO level → log noise hides real issues.
• Forgetting context – include IDs, user, request path for correlation.

```python
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

def process(order_id):
    logging.info('start order %s', order_id)
    # ... do work ...
    logging.info('finish order %s', order_id)

process('A12')
```

### Quick check

1. Which level is best for noisy loop diagnostics?
  a. DEBUG   b. WARNING

2. True / False Writing errors to `print()` is equivalent to proper logging.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – logging provides levels, timestamps, handlers.

</details>

## 27. Designing with basic UML: class & sequence sketches

UML diagrams act as **visual shorthand**.  Two light flavours worth knowing:

* **Class diagram** – boxes with class name, key attrs, and methods; arrows show is‑a or has‑a relationships.
* **Sequence diagram** – vertical lifelines and horizontal messages; clarifies who calls whom and order of operations.

Use a whiteboard version – no need for full tool suite.  The goal is *shared understanding*, not pixel‑perfect art.

```text
Class diagram sketch
  +------------+      uses      +------------+
  |  Checkout  |--------------->|  Payment   |
  +------------+                +------------+
  | - cart     |                | - gateway  |
  | + pay()    |                | + charge() |
  +------------+                +------------+

Sequence (pay)
Checkout -> Payment: charge()
Payment  -> Gateway: HTTP POST
Gateway  --> Payment: 201 OK
Payment  --> Checkout: success
```

### Quick check

1. True / False A UML class arrow with a hollow triangle denotes inheritance.

2. Sequence diagrams depict:
  a. data schema   b. call order

<details><summary>Answer key</summary>

1. **True**.
2. **b**.

</details>

## 28. Simple design patterns “lite”: factory, adapter, strategy, decorator

* **Factory**: centralises object creation, hides concrete classes.
* **Adapter**: wraps incompatible interface to match expected one.
* **Strategy**: interchangeable algorithms behind common interface.
* **Decorator**: adds behaviour around object without modifying original.

Think of them as *vocabulary words* for common shapes, not rigid templates.

```python
# Strategy example: different shipping cost algorithms
class ShippingStrategy:
    def cost(self, order): ...

class FlatRate(ShippingStrategy):
    def cost(self, _): return 5

class ByWeight(ShippingStrategy):
    def cost(self, order): return order.kg * 1.2

def checkout(order, strategy: ShippingStrategy):
    return order.total + strategy.cost(order)
```

### Quick check

1. Decorator pattern is mainly for:
  a. creation   b. augmentation

2. True / False A factory eliminates the need for dependency inversion.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – factory complements but doesn’t replace inversion.

</details>

## 29. Layered architecture: UI → service → data access

Classic 3‑layer stack:
• **UI** – presentation, controllers, API endpoints.
• **Service/domain** – business rules.
• **Data access** – persistence, external systems.

Each layer depends *only downward*.  Benefits: swap UI (web ↔ CLI) or storage (SQL ↔ in‑memory) without touching core logic.

```text
Request -> Flask route (UI)
        -> InvoiceService.generate() (service)
        -> InvoiceRepo.save() (data access)
```

### Quick check

1. True / False Data access layer may import UI layer utilities.

2. Changing DB driver should affect primarily:
  a. service layer   b. data layer

<details><summary>Answer key</summary>

1. **False** – violates downward dependency rule.
2. **b**.

</details>

## 30. Hexagonal / ports‑and‑adapters view

Hexagonal architecture (a.k.a. Ports & Adapters) puts **domain logic at the core**.  External actors (UIs, databases, message brokers) interact through **ports** (interfaces) implemented by **adapters**.

Advantages over straight layers:
• Core is **framework‑agnostic** – easier to unit‑test and migrate tech.
• Multiple adapters can plug into same core (REST, CLI, scheduler).

Key idea: dependencies point *inward*; outer parts know core, core knows only ports.

```text
           +-------------+
CLI ---->  |             |  <---- DB Adapter
HTTP --->  |  DOMAIN     |  <---- Email Adapter
           +-------------+
```

### Quick check

1. Ports reside in:
  a. domain core   b. outer adapter

2. True / False Hexagonal allows replacing REST API with gRPC without touching domain logic.

<details><summary>Answer key</summary>

1. **a** – interfaces live in core.
2. **True**.

</details>